<a href="https://colab.research.google.com/github/srinivasdasu24/__DataScience/blob/main/CrossSell_Prediction/InClass_Working_Code_5_Combine_Missing%2C_Outlier_and_Transformation_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code 5
- Combine all Strategy
  - Missing Value Imputation
  - Outlier Startegy
  - Tranformation Strategy

## 1/ Import Libraries

In [1]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [2]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [3]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction/input/test.csv')

## 3/ Combine both Datasets (train, test)

In [4]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [5]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

id                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
Response                127037
dtype: int64

In [6]:
#CodeSection6
# Check Data types for each variable
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


## 4/ Numerical Processing

### 4.1/ Impute Numerical Variables

In [7]:
#CodeSection7
# Identify all Numerical features
numerical_features = ['Age', 'Driving_License', 'Region_Code', 'Previously_Insured','Annual_Premium', 'Policy_Sales_Channel','Vintage']

In [8]:
#CodeSection8
# Import SimpleImputer
from sklearn.impute import SimpleImputer

In [9]:
#CodeSection9

#Initiate SimpleImputer
num_impute = SimpleImputer(strategy='median')

# Fit SimpleImputer on Numerical Features
num_impute.fit(train[numerical_features])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [10]:
#CodeSection10
# Impute Train and Test
for data in all_data:
  data[numerical_features] = num_impute.transform(data[numerical_features])

### 4.2/ Outlier Strategy

In [11]:
#CodeSection11

# Write a loop to detect and impute Outliers
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  for dataset in all_data:
    dataset[num_var] = dataset[num_var].apply(lambda x : Upper_Whisker if x > Upper_Whisker else x)

### 4.3/ Transformation Strategy

In [12]:
#CodeSection12
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

In [13]:
#CodeSection13

#Initiate StandardScaler
num_scale = StandardScaler()

# Fit StandardScaler
num_scale.fit(data[numerical_features])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
#CodeSection14
# Scale Train and Test
for data in all_data:
  data[numerical_features] = num_scale.transform(data[numerical_features])

## 5/ Impute Categorical Variables

### Impute all Categorical Variables at once

In [15]:
#CodeSection15
# Identify all Categorical Features
categorical_features = ['Gender', 'Vehicle_Damage', 'Vehicle_Age']

In [16]:
#CodeSection16
# Import SimpleImputer

In [17]:
#CodeSection17

#Initiate SimpleImputer
cat_impute = SimpleImputer(strategy='most_frequent')

# Fit SimpleImputer
cat_impute.fit(data[categorical_features])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [18]:
#CodeSection18
# Impute Train and Test
for data in all_data:
  data[categorical_features] = cat_impute.transform(data[categorical_features])

## 6/ Now Re Check all Features

In [19]:
#CodeSection19
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

id                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
Response                127037
dtype: int64

## 8/ Prepare Data for Model Model

In [20]:
#CodeSection20
# Check First 5 Rows of Test Data
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,-0.890089,0.043233,-1.170327,1.083387,< 1 Year,No,0.365324,0.739349,-1.211054
1,381111,Male,0.079795,0.043233,0.116590,-0.923031,1-2 Year,Yes,0.234597,-1.927487,-0.517782
2,381112,Male,0.532408,0.043233,0.116590,-0.923031,1-2 Year,Yes,0.640728,0.224373,0.534079
3,381113,Male,-0.954748,0.043233,0.040889,1.083387,< 1 Year,Yes,0.466728,0.739349,0.390643
4,381114,Male,-0.760771,0.043233,0.116590,1.083387,< 1 Year,No,1.870940,0.739349,1.705469


In [21]:
#CodeSection21
# Get X and y Variables for your Dataset
X = train.drop(['id','Response'], axis =  1)
y = train['id']

### Create Dummy Variables for  Categorical Variables

In [22]:
#CodeSection22
# Get Dummy Variables
X = pd.get_dummies(X, drop_first = True)  # Keep only n - 1 Variables - Use "drop_first = True"

## 9/ Build Model

In [23]:
#CodeSection23
#Import Logistic Regression Library
from sklearn.linear_model import LogisticRegression

#Initialize LR
clf = LogisticRegression(max_iter = 500)

In [ ]:
#CodeSection24
# Fit Model
model = clf.fit(X, y)

### Check Accuracy of Model on Train Data

In [ ]:
#CodeSection25
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [ ]:
#CodeSection26
# Import metrics library
from sklearn.metrics import accuracy_score

In [ ]:
#CodeSection27
# get Actual "y" variables (use "y_true" as variable)
y_true = train['Response']

In [ ]:
#CodeSection28
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

## 10/ Predict and Submit

### Predict

In [ ]:
#CodeSection29
# Get all the X Variables from the Test Dataset
X_test = test.drop(['id'], axis =  1)

In [ ]:
#CodeSection30
#Covert to Dummies
X_test = pd.get_dummies(X_test, drop_first = True)

In [ ]:
#CodeSection31
# use this code if the next Code shows error
#X_test['Married_Missing']  = 0

In [ ]:
#CodeSection32
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [ ]:
#CodeSection33
submission = pd.DataFrame({
    'id' : test['id'],
    'Response' : X_test_prep
})

### Export Submission File

In [ ]:
#CodeSection34
submission.to_csv('/mntDrive/MyDrive/__DataScience/2 Cross Sell Prediction/output/O5_Preproccesing_All.csv', index = False)

In [ ]:
a = []
while 1:
    a.append("1")

In [ ]:
# LB Accuracy : 0.7777